In [10]:
#| echo: false
import pandas as pd  
import numpy as np  
from scipy.stats import pearsonr, pointbiserialr, chi2_contingency  
import matplotlib.pyplot as plt  
from matplotlib.font_manager import FontProperties  

# 加载数据  
file_path = 'data/listings.csv'    # 使用用户提供的文件路径  
airbnb_data = pd.read_csv(file_path)  

# 查看数据基本信息  
print(airbnb_data.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91778 entries, 0 to 91777
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              91778 non-null  int64  
 1   name                            91778 non-null  object 
 2   host_id                         91778 non-null  int64  
 3   host_name                       91773 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   91778 non-null  object 
 6   latitude                        91778 non-null  float64
 7   longitude                       91778 non-null  float64
 8   room_type                       91778 non-null  object 
 9   price                           87598 non-null  float64
 10  minimum_nights                  91778 non-null  int64  
 11  number_of_reviews               91778 non-null  int64  
 12  last_review                     

In [11]:
# Calculate the estimation of nights booked for each listing
airbnb_data = airbnb_data[airbnb_data['availability_365'] > 0] 
airbnb_data['estimated_nights_booked'] = airbnb_data['reviews_per_month'] * 12 * airbnb_data['minimum_nights'] * 2 

In [13]:
#Data cleaning
# Replace NaN with 0
airbnb_data['estimated_nights_booked'] = airbnb_data['estimated_nights_booked'].fillna(0)

# Convert the column to integers
airbnb_data['estimated_nights_booked'] = airbnb_data['estimated_nights_booked'].astype(int)

# Filter the DataFrame to include only rows where estimated_nights_booked is greater than 90
filtered_data = airbnb_data[airbnb_data['estimated_nights_booked'] > 90]

#Export to .csv
filtered_data.to_csv('cleaned_listings.csv', index=False)